In [ ]:
import os
import cv2
import numpy as np
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, Sequential

In [ ]:
def load_xnn_model():
    base_model = InceptionV3(weights='imagenet', include_top=True)
    
    #print(base_model.summary())
    #print base_model.get_layer('global_max_pooling2d_1').output
    outputs = base_model.get_layer('avg_pool').output

    xnn_model = Model(inputs=base_model.input, outputs=outputs)

    return xnn_model

In [ ]:
xnn_model=load_xnn_model()

In [ ]:
from keras.models import load_model
rnn_model = load_model("lrcnn-6class-model.h5")

In [ ]:
path='latest_frames'
folders=os.listdir(path)
X=[]
for folder in folders:
    fpath=os.path.join(path,folder)
    all_frames=os.listdir(fpath)
    
    frames=[]
    for frame in all_frames:
        image=cv2.imread(os.path.join(fpath,frame))
        #image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image=cv2.resize(image,(480,640))
        frames.append(image)
        
    sequence = []
    for img in frames:
        #x = image.img_to_array(img)
        x = np.expand_dims(img, axis=0)
        x = preprocess_input(x)
        features = xnn_model.predict(x) # for some reason not on graph?
        #print(x)
        #print(features[0])
        sequence.append(features[0])
        
    X.append(sequence)
    
X=np.array(X)
print(X.shape)
results=rnn_model.predict_classes(X)

for i,result in zip(folders,results):
    if result==0:
        x='Backhand'
    elif result==1:
        x='BVolley'
    elif result==2:
        x='Forehand'
    elif result==3:
        x='FVolley'
    elif result==4:
        x='Service'
    else:
        x='Smash'
    print(i,'-',x)